# Imports

In [1]:
import sys
 
# setting path
sys.path.append('..')

In [2]:
from dataset_builder import EEDatasetBuilder
import geemap
import ee

# Map

In [3]:
Map = geemap.Map(center=[-11.8673, -76.9544], zoom=2.5)
Map

Map(center=[-11.8673, -76.9544], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(c…

In [4]:
# globe = ee.FeatureCollection('projects/ee-margauxmasson21-shapefiles/assets/world_rectangle')
# Map.addLayer(globe.geometry())

# Dataset builder

In [5]:
ee_dataset_builder = EEDatasetBuilder()

Make the biomass band with some filters

In [6]:
ee_dataset_builder.filtered_biomass_layer_from_raster(
    biomass_raster='Spawn_AGB_tCO2e',
    filter_dict={'forest_non_forest': {'date_range':['2010-01-01', '2010-12-31']},
                 'min_forest_age': {'age':40},
                # 'very_low_density_rural': {'year': 2010},
                # 'forest_loss': {'year': 2010, 'distance': 5000},
                # 'forest_gain' : {'distance': 5000},
                # 'roads': {'distance': 5000},
                # 'fire': {'year': 2010}
    }
)

Make covariates bands

In [7]:
ee_dataset_builder.spatial_covariates(covariates=['ecoregion', 'terrain'])

List band names of image

In [8]:
ee_dataset_builder.image.bandNames().getInfo()

['Spawn_AGB_tCO2e', 'BIOME_NUM', 'elevation', 'aspect', 'slope', 'hillshade']

In [9]:
ee_dataset_builder.image

So the image has the added bands; it basically seems to work.

Use the below code to make a plot and confirm results of mask match what is expected for both biomass and the features.

### Visualize in map

In [10]:
biomass_vis_param = { 
  'min': 0.0,
  'max': 600.0,
  'palette': ['d9f0a3', 'addd8e', '78c679', '41ab5d', '238443', '005a32']}

Map.addLayer(ee_dataset_builder.image.select('Spawn_AGB_tCO2e'), biomass_vis_param, 'Mature Forest Biomass', True)

In [11]:
biome_vis_param = { 
  'min': 0.0,
  'max': 50.0,
  'palette':  [
    "1c5510","659208","ae7d20","000065","bbcb35","009a18",
    "caffca","55eb49","65b2ff","0020ca","8ea228","ff9adf",
    "baff35","ffba9a","ffba35","f7ffca","e7e718","798649",
    "65ff9a","d29e96",
  ]};

Map.addLayer(ee_dataset_builder.image.select('BIOME_NUM'), biome_vis_param, 'Biome Num', True)

In [12]:
# asset_img = ee.Image('projects/ee-mmf-mature-forest-biomass/assets/mature_forest_2010_AGB_tco2_scale_100')
# Map.addLayer(asset_img, biomass_vis_param, 'Asset Mature Forest Biomass', True)

## Export samples CSV to Google Drive

Check progress in your GEE console: https://code.earthengine.google.com/tasks

In [13]:
# Gridded world shapefile asset in GEE
shp_asset_path = 'projects/ee-margauxmasson21-shapefiles/assets/latin_america_gridded_5000km2'

# This will take quite some time
ee_dataset_builder.samples_csv_export(shp_asset_path, 
                                    name_output_google_drive_folder='test_export_samples_latinAm_gridded_5000km2', 
                                    numPixels=2000, scale=100)

## Export tiles to Google Drive

In [ ]:
shp_asset_path = 'projects/ee-margauxmasson21-shapefiles/assets/latin_america_gridded_10degrees'
name_google_drive_folder = 'test_export_tiles_latinAm_gridded_10degrees'
maxPixels = 1e13
scale=100

# This will take quite some time
ee_dataset_builder.tiles_export(shp_asset_path, 
                                    name_output_google_drive_folder=name_google_drive_folder, 
                                    scale=scale, maxPixels=maxPixels)